In [2]:
import json
import pickle
import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences 

import time
from nltk.tokenize import word_tokenize

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopset = set(stopwords.words('english'))

import pandas as pd
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [4]:
hp_data = pd.read_csv('hp_all_comments.csv')
hp_data.comment=hp_data.comment.astype(str)

In [15]:
all_data = hp_data['comment'][:1000000]

TOP_N = 1000
hp_tok = Tokenizer(num_words = TOP_N, oov_token='<unk>')
hp_tok.fit_on_texts(all_data)

In [16]:
len(hp_tok.word_index) #vocab is too large!

285460

In [17]:
top_words = sorted(hp_tok.word_counts.items(), key=lambda x:x[1])[::-1][:TOP_N]

In [18]:
mini_word_dict = {} #{word : word_freq}
for word in top_words:
    mini_word_dict[word[0]] = word[1]





"""
below code too slow!
"""
new_data = []
for orig_comment in all_data:
    new_comment = []
    for word in text_to_word_sequence(orig_comment):
        if word not in top_words:
            new_comment.extend('<oov>')
        else:
            new_comment.extend(word)
    new_data.append(new_comment)

In [11]:
def transform_text(old_comment, word_dict):
    tokenized = text_to_word_sequence(old_comment)
    new_comment = ''
    for word in tokenized:
        if word not in word_dict:
            new_comment = new_comment + ' <oov> '
        else:
            new_comment = new_comment+ ' '+word+ ' '
    return new_comment

In [12]:
transform_text(all_data[0],mini_word_dict)

' <oov>  <oov>  that  <oov>  the  hell  out  of  me  christians  what  bible  are  they  reading '

In [19]:
from functools import partial

start = time.time()
mapfunc = partial(transform_text, word_dict=mini_word_dict)
new_all_data= list(map(mapfunc, all_data))
print('elasped time: {}'.format(time.time()-start))

elasped time: 16.258278131484985


In [20]:

with open('clean_hp_1k_comments.json', 'w') as outfile:
    json.dump(new_all_data, outfile)